# Forest CoverType 2b): PyTorch TabNet

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import json
import os

# External Dependencies:
import boto3
from botocore import exceptions as botoexceptions
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.pytorch.estimator import PyTorch as PyTorchEstimator
from sagemaker.pytorch.model import PyTorchModel
import seaborn as sn
from sklearn import metrics
from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

# Local Dependencies:
import util

In [ ]:
%store -r bucket_name
%store -r experiment_name
%store -r preproc_trial_component_name

bucket = boto3.resource("s3").Bucket(bucket_name)
role = sagemaker.get_execution_role()
smclient = boto3.client("sagemaker")
smsess = sagemaker.session.Session()

In [ ]:
tabnet_trial = Trial.create(
    trial_name=util.append_timestamp("tabnet-pytorch"), 
    experiment_name=experiment_name,
    sagemaker_boto_client=smclient,
)
tabnet_trial.add_trial_component(preproc_trial_component_name)

preproc_trial_component = TrialComponent.load(preproc_trial_component_name)

In [ ]:
hyperparameters = {
    "model-type": "classification",
    "target": "Cover_Type",
    "seed": 1337,
    "n-d": 64,
    "n-a": 64,
    "n-steps": 5,
    "lr": 0.02,
    "gamma": 1.5,
    "n-independent": 2,
    "n-shared": 2,
    #"cat-idxs": ",".join(map(lambda i: str(i), cat_idxs)),
    # cat-dims???
    #"cat-emb-dim": ",".join(map(lambda i: str(i), cat_emb_dim)),
    "lambda-sparse": 1e-4,
    "momentum": 0.3,
    "clip-value": 2.,
    "max-epochs": 500,  # Try 1000 for accuracy
    "patience": 100,
    "batch-size": 16384,
    "virtual-batch-size": 256,
    "num-workers": 2,
}


estimator = PyTorchEstimator(
    role=role,
    entry_point="train.py",
    source_dir="src",
    framework_version="1.4",

    base_job_name="forestcover-tabnet",
    output_path=f"s3://{bucket_name}/trainjobs",
    checkpoint_s3_uri=f"s3://{bucket_name}/trainjobs",

    debugger_hook_config=False,

    train_instance_count=1,
    train_instance_type="ml.p3.2xlarge",
    hyperparameters=hyperparameters,
    metric_definitions=[
        # One console log per output e.g.:
        # | EPOCH | train | valid | total time (s)
        # | 1 | 0.58782 | 0.06811 | 25.5
        # Since these rows are a bit brusque, we'll write quite precise/picky regexs to stay safe:
        { "Name": "train:accuracy", "Regex": r"\| +\d+ +\| +(.*?) +\| +[^\s]+ +\| +[^\s]+", },
        { "Name": "validation:accuracy", "Regex": r"\| +\d+ +\| +[^\s]+ +\| +(.*?) +\| +[^\s]+", },
    ],
    enable_sagemaker_metrics=True,
)

In [ ]:
estimator.fit(
    inputs={
        "train": preproc_trial_component.output_artifacts["train-csv"].value,
        "validation": preproc_trial_component.output_artifacts["validation-csv"].value,
    },
    experiment_config={
        # This will create a TrainingJob-linked TrialComponent and automatically attach hyperparameters etc
        "TrialName": tabnet_trial.trial_name,
        "TrialComponentDisplayName": "Training",
    },
    #wait=False,
)

## Test

In [ ]:
training_job_desc = estimator.latest_training_job.describe()
model_path = training_job_desc["ModelArtifacts"]["S3ModelArtifacts"]
model_name = training_job_desc["TrainingJobName"]

In [ ]:
try:
    model.delete_model()
except (AttributeError, NameError, ValueError):
    # AttributeError: Model() wasn't initialized with a sagemaker_session and hasn't created one yet
    # NameError: model hasn't been defined yet
    # ValueError: Current model isn't saved to SageMaker
    pass
except botoexceptions.ClientError as e:
    if (
        e.response["Error"]["Code"] == "ValidationException"
        and e.response["Error"]["Message"].startswith("Could not find")
    ):
        # SDK tried to delete but model wasn't found
        pass
    else:
        raise e

model = PyTorchModel(
    name=model_name,
    model_data=model_path,
    role=role,
    source_dir="src/",
    entry_point="src/inference.py",
    framework_version="1.4",
    sagemaker_session=smsess,
)

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.p3.2xlarge",  # g4dn family not yet supported for batch transform
    strategy="MultiRecord",
    max_payload=1,  # 1MB
    accept="text/csv",  # Need to specify input and output types when using filters
    assemble_with="Line",
    output_path=f"s3://{bucket_name}/test/{model.name}",
)

In [ ]:
transformer.transform(
    preproc_trial_component.output_artifacts["test-csv"].value,
    split_type="Line",
    content_type="text/csv",  # Need to specify input and output types when using filters
    # TODO: Check why -2 is required vs -1 per JSONPath spec for trimming last column
    input_filter="$[:-2]",  # Exclude target column from input to the model
    join_source="Input",  # Store both input and output in the result (saves us re-joining in notebook)
    # No output_filter so our output will be all source columns (incl target) + all prediction columns
    experiment_config={
        "ExperimentName": experiment_name,
        "TrialName": tabnet_trial.trial_name,
        "TrialComponentDisplayName": "Test",
    },
    wait=True,
    logs=True,
)

In [ ]:
test_root_filename = preproc_trial_component.output_artifacts["test-csv"].value.rpartition("/")[2]

!mkdir -p data/test/$model_name
bucket.download_file(
    f"test/{model.name}/{test_root_filename}.out",  # Batch Transform appends ".out"
    f"data/test/{model.name}/{test_root_filename}",
)

In [ ]:
with open("data/columns.json", "r") as f:
    train_columns = json.load(f)

# TODO: Save from data prep
# Note our first cover_type is a dummy because the dataset's encoding starts at 1.
cover_types = ("N/A", "Spruce/Fir", "Lodgepole Pine", "Ponderosa Pine", "Cottonwood/Willow", "Aspen", "Douglas-fir", "Krummholz")

result_cols = train_columns[:-1] + ["Actual_Cover_Type"] + ["Pred " + typ for typ in cover_types[1:]]

In [ ]:
df_test_results = pd.read_csv(
    f"data/test/{model.name}/{test_root_filename}",
    names=result_cols
)

print(f"Shape: {df_test_results.shape}")
df_test_results.head()

In [ ]:
# Results just have class probabilities: Recover the predicted class column (argmax):
df_pred_probs = df_test_results[["Pred " + typ for typ in cover_types[1:]]]
# idxmax axis=1 returns index values (i.e. column names) - so rename columns to numbers:
predicted_classes = df_pred_probs.rename(
    columns={x:y for x,y in zip(df_pred_probs.columns,range(1,1+len(df_pred_probs.columns)))}
).idxmax(axis=1)

# Now we can add the post-processed results into the main dataframe:
df_test_results["Pred_Cover_Type"] = predicted_classes
df_test_results["Pred_Correct"] = df_test_results["Pred_Cover_Type"] == df_test_results["Actual_Cover_Type"]
df_test_results.head()

In [ ]:
# Plot confusion matrix and accuracy summary:

confusion = metrics.confusion_matrix(df_test_results["Actual_Cover_Type"], df_test_results["Pred_Cover_Type"])

plt.figure(figsize = (10,7))
sn.heatmap(
    pd.DataFrame(
        confusion,
        index = cover_types[1:],
        columns = cover_types[1:],
    ),
    annot=True
)

n_correct = sum(df_test_results["Pred_Correct"])
n_tested = len(df_test_results)
print(f"{n_correct} of {n_tested} samples correct: Accuracy={n_correct/n_tested:.3%}")

In [ ]:
# TODO: Interactive ROC curve?

In [ ]:
# TODO: Associate accuracy metric with existing Test TrialComponent?

# It's not trivial because neither training nor processing jobs seem to advertise what TrialComponentName
# they created during run: We only have access to the DisplayName we requested... So could work around by
# appending timestamps to our displayname (to make lookups unique when job retried multiple times within
# trial), but that would make the structure within a trial less straightforward.

## Deploy Demo

To be moved to deployment section

In [ ]:
try:
    predictor.delete_endpoint()
except NameError:
    pass

predictor = model.deploy(
    endpoint_name=model.name,  # Use model.name to avoid us accidentally deploying the model twice
    initial_instance_count=1,
    instance_type="ml.g4dn.xlarge",
    #wait=False
)

In [ ]:
df_test = pd.read_csv(
    "data/test-noheader.csv",
    names=train_columns
)

In [ ]:
# By default, PyTorch predictors use serializer sagemaker.predictor.npy_serializer and content_type
# application/x-npy ...So it's as easy as dropping the target column and converting pandas->numpy.
# (Note you can do many more than 10 samples at once - see the batch transform logs)
result = predictor.predict(df_test.drop("Cover_Type", axis=1).iloc[0:10].to_numpy())
result